In [7]:
import time
import pandas as pd
import numpy as np
import ccxt
from binance.client import Client
from datetime import datetime

In [2]:
api_key = 'dNC0absMhm1yaOQ5dlG9xQabyqIzpNdJo9PJ6l7snb7O17DVWWPtHvQ5GUpgm0Tq'
api_secret = 'Xs5Lo26gAAJrAN6Lg4FS9KBkJvKl7zjMP5JRxwTs9IKt1drdLopug6TS8h1asuc9'

client = Client(api_key, api_secret)

In [3]:
# Initialize Variables
CANDLE_DURATION_IN_MIN = 5

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
SMA_FAST_PERIOD = 5
SMA_SLOW_PERIOD = 10

CCXT_TICKER_NAME = 'BTC/USDT'
TRADING_TICKER_NAME = 'btcusdt'

INVESTMENT_AMOUNT_DOLLARS = 10
HOLDING_QUANTITY = 0

exchange = ccxt.binance()
binance_client = Client(api_key, api_secret)

In [6]:
fetch_data(CCXT_TICKER_NAME)

,at,open,high,low,close,vol,Date,symbol
0,1681204800000,30017.14,30032.05,29974.00,30006.61,355.95392,2023-04-11 09:20:00,BTC/USDT
1,1681205100000,30006.61,30061.15,30006.61,30056.75,169.67993,2023-04-11 09:25:00,BTC/USDT
2,1681205400000,30056.76,30079.99,30056.75,30062.70,137.97331,2023-04-11 09:30:00,BTC/USDT
3,1681205700000,30062.70,30063.49,30049.56,30055.52,106.86664,2023-04-11 09:35:00,BTC/USDT
4,1681206000000,30055.51,30078.40,30015.06,30015.06,168.25442,2023-04-11 09:40:00,BTC/USDT
...,...,...,...,...,...,...,...,...
94,1681233000000,30118.87,30141.11,30114.44,30115.40,139.68954,2023-04-11 17:10:00,BTC/USDT
95,1681233300000,30115.40,30119.69,30100.00,30105.48,99.07636,2023-04-11 17:15:00,BTC/USDT
96,1681233600000,30105.47,30124.50,30040.00,30124.50,292.98820,2023-04-11 17:20:00,BTC/USDT
97,1681233900000,30124.50,30156.57,30124.49,30155.01,127.02536,2023-04-11 17:25:00,BTC/USDT


In [4]:
def rsi(data, period):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [5]:
def macd(data, slow=26, fast=12, signal_period=9):
    exp1 = data['close'].ewm(span=fast, adjust=False).mean()
    exp2 = data['close'].ewm(span=slow, adjust=False).mean()
    macd_line = exp1 - exp2
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    return macd_line, signal_line

In [4]:
def fetch_data(ticker):
    global exchange
    bars, ticker_df = None, None

    try:
        bars = exchange.fetch_ohlcv(ticker, timeframe=f'{CANDLE_DURATION_IN_MIN}m', limit=100)
    except:
        print(f"Error in fetching data from the exchange:{ticker}")

    if bars is not None:
        ticker_df = pd.DataFrame(bars[:-1], columns=['at', 'open', 'high', 'low', 'close', 'vol'])
        ticker_df['Date'] = pd.to_datetime(ticker_df['at'], unit='ms')
        ticker_df['symbol'] = ticker

    return ticker_df

In [7]:
def get_trade_recommendation(ticker_df):
    close_prices = ticker_df['close']
    rsi = close_prices.diff().apply(lambda x: x if x > 0 else 0).rolling(window=RSI_PERIOD).mean() / close_prices.diff().abs().rolling(window=RSI_PERIOD).mean()
    rsi = 100 - (100 / (1 + rsi))

    sma_fast = close_prices.rolling(window=SMA_FAST_PERIOD).mean()
    sma_slow = close_prices.rolling(window=SMA_SLOW_PERIOD).mean()

    rsi_buy = rsi.iloc[-1] <= RSI_OVERSOLD
    rsi_sell = rsi.iloc[-1] >= RSI_OVERBOUGHT

    sma_crossover = sma_fast.iloc[-1] > sma_slow.iloc[-1] and sma_fast.iloc[-2] <= sma_slow.iloc[-2]
    sma_crossunder = sma_fast.iloc[-1] < sma_slow.iloc[-1] and sma_fast.iloc[-2] >= sma_slow.iloc[-2]

    if rsi_buy or sma_crossover:
        return 'BUY'
    elif rsi_sell or sma_crossunder:
        return 'SELL'
    else:
        return 'WAIT'

In [8]:
def execute_trade(trade_rec_type, trading_ticker):
    global binance_client, HOLDING_QUANTITY
    try:
        ticker_price_response = binance_client.get_symbol_ticker_price(symbol=trading_ticker)
        current_price = float(ticker_price_response['price'])

        side_value = 'buy' if trade_rec_type == "BUY" else 'sell'
        scrip_quantity = round(INVESTMENT_AMOUNT_DOLLARS / current_price, 5) if trade_rec_type == "BUY" else HOLDING_QUANTITY

        print(f"PLACING ORDER {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}: "
              f"{trading_ticker}, {side_value}, {current_price}, {scrip_quantity}, {int(time.time() * 1000)} ")

        order_response = binance_client.create_order(
            symbol=trading_ticker,
            side=side_value.upper(),
            type="MARKET",
            quantity=scrip_quantity
        )

        print(f"ORDER PLACED")
        HOLDING_QUANTITY = scrip_quantity if trade_rec_type == "BUY" else 0
        return True
    except Exception as e:
        print(f"\nALERT!!! UNABLE TO COMPLETE ORDER\n{e}")
        return False

In [9]:
def run_bot_for_ticker(ccxt_ticker, trading_ticker):
    currently_holding = False
    while True:
        ticker_data = fetch_data(ccxt_ticker)
        if ticker_data is not None:
            trade_rec_type = get_trade_recommendation(ticker_data)
            print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}  TRADING RECOMMENDATION: {trade_rec_type}')

            if (trade_rec_type == 'BUY' and not currently_holding) or \
                    (trade_rec_type == 'SELL' and currently_holding):
                print(f'Placing {trade_rec_type} order')
                trade_successful = execute_trade(trade_rec_type, trading_ticker)
                currently_holding = not currently_holding if trade_successful else currently_holding

            time.sleep(CANDLE_DURATION_IN_MIN * 60)
        else:
            print(f'Unable to fetch ticker data - {ccxt_ticker}. Retrying!!')
            time.sleep(5)

In [12]:
run_bot_for_ticker(CCXT_TICKER_NAME, TRADING_TICKER_NAME)

10/04/2023 20:47:50  TRADING RECOMMENDATION: WAIT


KeyboardInterrupt: 

In [ ]:
fetch_data(CCXT_TICKER_NAME)